In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np


In [25]:
df = pd.read_csv("Metadata.csv")
df.head
df.columns


Index(['Piece Title', 'Genre', 'Bpm', 'Key', 'Time Signature', 'Mode Ionian',
       'Duration in seconds', 'Instrument 1', 'Instrument 2', 'Instrument 3',
       'Instrument 4', 'Instrument 5', 'Mood 1', 'Mood 2', 'Mood 3', 'Mood 4',
       'Mood 5', 'Production Style', 'Overall Quality',
       'TV PLACEMENT YES: TARGET'],
      dtype='object')

In [26]:
target_col = 'TV PLACEMENT YES: TARGET'
numerical_cols = ['Bpm', 'Duration in seconds',]
categorical_cols = ['Genre', 'Key', 'Time Signature', 'Mode Ionian', 'Instrument 1', 'Instrument 2', 'Instrument 3', 'Instrument 4', 'Instrument 5', 'Mood 1', 'Mood 2', 'Mood 3', 'Mood 4', 'Mood 5', 'Production Style', 'Overall Quality',]
df = df.dropna(subset=numerical_cols + categorical_cols + [target_col])


In [27]:
scaler = StandardScaler()
X_num = scaler.fit_transform(df[numerical_cols])

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le


In [29]:

X = df[numerical_cols + categorical_cols]
y = df[[target_col]]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(type(X))
print(type(y))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [30]:
#dataset dataloader

class MusicDataset(Dataset):
    def __init__(self, X, y, categorical_cols, numerical_cols):
        self.cat_data = torch.tensor(X[categorical_cols].values, dtype=torch.long)
        self.num_data = torch.tensor(X[numerical_cols].values, dtype=torch.float32)
        self.labels = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.cat_data[idx], self.num_data[idx], self.labels[idx]




In [47]:
embedding_sizes = [
    (df[col].nunique(), min(50, (df[col].nunique() + 1) // 2))
    for col in categorical_cols
]
def train_model(model, dataloader, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_cat, x_num, y in dataloader:
            optimizer.zero_grad()
            output = model(x_cat, x_num)
            y = y.view(-1, 1)  # reshape target to (batch_size, 1)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")



In [57]:


class MusicClassifier(nn.Module):

    def __init__(self, embedding_sizes, num_numerical):
        super().__init__()
        
        # Create embedding layers for categorical features
        self.embeddings = nn.ModuleList([
            nn.Embedding(categories, size) for categories, size in embedding_sizes
        ])
        
        emb_dim = sum(e.embedding_dim for e in self.embeddings)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(emb_dim + num_numerical, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)  # binary output
        
        self.dropout = nn.Dropout(0.5)  # optional dropout
        
    def forward(self, cat_data, num_data):
        # Apply embeddings on categorical columns
        embedded = [emb(cat_data[:, i]) for i, emb in enumerate(self.embeddings)]
        x_cat = torch.cat(embedded, dim=1)  # concatenate all embeddings
        
        # Concatenate embedded categorical with numerical features
        x = torch.cat([x_cat, num_data], dim=1)
        
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.output(x)
        x = torch.sigmoid(x) # probabilities between 0 and 1
        return x


In [58]:
train_dataset = MusicDataset(X_train, y_train, categorical_cols, numerical_cols)
test_dataset = MusicDataset(X_test, y_test, categorical_cols, numerical_cols)

# Define dataset
X = df[categorical_cols + numerical_cols]
y = df[target_col]

dataset = MusicDataset(X, y, categorical_cols, numerical_cols)


# Define model
model = MusicClassifier(embedding_sizes, num_numerical=len(numerical_cols))

# Train
train_model(model, dataloader, epochs=200)

torch.save(model.state_dict(), "model.pt")

Epoch 1, Loss: 3.3521
Epoch 2, Loss: 1.8365
Epoch 3, Loss: 1.7755
Epoch 4, Loss: 1.7116
Epoch 5, Loss: 1.5037
Epoch 6, Loss: 1.4812
Epoch 7, Loss: 1.6121
Epoch 8, Loss: 1.4280
Epoch 9, Loss: 1.5646
Epoch 10, Loss: 1.2746
Epoch 11, Loss: 1.3500
Epoch 12, Loss: 1.2603
Epoch 13, Loss: 1.2610
Epoch 14, Loss: 1.2592
Epoch 15, Loss: 1.0718
Epoch 16, Loss: 1.4237
Epoch 17, Loss: 1.1888
Epoch 18, Loss: 1.1642
Epoch 19, Loss: 1.1279
Epoch 20, Loss: 1.1278
Epoch 21, Loss: 1.0548
Epoch 22, Loss: 1.1496
Epoch 23, Loss: 0.8853
Epoch 24, Loss: 1.1007
Epoch 25, Loss: 0.8887
Epoch 26, Loss: 0.7890
Epoch 27, Loss: 0.8140
Epoch 28, Loss: 0.9981
Epoch 29, Loss: 0.8025
Epoch 30, Loss: 0.6382
Epoch 31, Loss: 0.7387
Epoch 32, Loss: 0.8050
Epoch 33, Loss: 0.7041
Epoch 34, Loss: 0.5837
Epoch 35, Loss: 0.4374
Epoch 36, Loss: 0.4522
Epoch 37, Loss: 0.5379
Epoch 38, Loss: 0.3122
Epoch 39, Loss: 0.2868
Epoch 40, Loss: 0.3036
Epoch 41, Loss: 0.3988
Epoch 42, Loss: 0.2404
Epoch 43, Loss: 0.2477
Epoch 44, Loss: 0.20

In [59]:
print(type(X_train))

model.load_state_dict(torch.load("model.pt"))
model.eval()

<class 'pandas.core.frame.DataFrame'>


MusicClassifier(
  (embeddings): ModuleList(
    (0): Embedding(12, 6)
    (1): Embedding(27, 14)
    (2): Embedding(5, 3)
    (3): Embedding(6, 3)
    (4): Embedding(25, 13)
    (5): Embedding(36, 18)
    (6): Embedding(31, 16)
    (7): Embedding(37, 19)
    (8): Embedding(24, 12)
    (9): Embedding(20, 10)
    (10-11): 2 x Embedding(22, 11)
    (12): Embedding(21, 11)
    (13): Embedding(10, 5)
    (14): Embedding(16, 8)
    (15): Embedding(4, 2)
  )
  (fc1): Linear(in_features=164, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [60]:
def evaluate_model(model, dataloader, device='cpu'):
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for cat_data, num_data, labels in dataloader:
            cat_data = cat_data.to(device)
            num_data = num_data.to(device)
            labels = labels.to(device)

            outputs = model(cat_data, num_data)
            preds = torch.sigmoid(outputs) 
            preds = (preds > 0.5).float()   # threshold at 0.5

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    accuracy = (all_preds == all_labels).float().mean().item()

    print(f"Test Accuracy: {accuracy:.4f}")
    return all_preds, all_labels

In [61]:
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds, labels = evaluate_model(model, test_dataloader, device=device)


Test Accuracy: 0.6963
